<img src = 'https://bancariosbh.org.br/manager/resources/uploads/213/350_198_8e66ca00d7dfdfbd00096a7c220be22c.jpg' height="200" width="340">
<p><b><font size = 5 face = 'verdana'> TAREFA AVALIATIVA 4 - PROCESSAMENTO DE LINGUAGEM NATURAL</font></b></p>

<p><font size = 3 face = 'verdana'>Trabalho apresentado à conclusão do curso de PROCESSAMENTO DE LINGUAGEM NATURAL do curso de Pós-Graduação em Inteligência Artificial, PUC Minas, como requisito parcial de avaliação. </font></b></p>
<p><font size = 3 face = 'verdana'><b>PROFESSORA:</b> BARBARA SILVEIRA FRAGA<br> 
<b>ALUNO:</b> OSEMAR DA SILVA XAVIER<br>
<b>MATRÍCULA:</b> 1418586</font></p>
<p><font size = 3 face = 'verdana'>
<hr></hr>

<b>TÓPICOS ABORDADOS:</b><br>
<li>SPACY;</li>
<li>POS TAGGING;</li>
<li>ABORDAGEM MANUAL;</li>
<li>MAC_MORPHO COM NLTK;</li>
<li>POS TAGGING COM SPACY - DESAMBIGUANDO FRASES;</li>
<li>NAMED ENTITY RECOGNITION;</li>
<li>RETREINO NER.</li>

In [2]:
!python -m spacy download pt
!python -m spacy download en
!pip install spacy==2.3
!pip install beautifulsoup4
!pip install pandas

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 20:47:14.424751: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'pt' are deprecated. Please use the
full pipeline package name 'pt_core_news_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.0 MB 520 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-16 20:47:30.859053: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As o

In [3]:
import pandas as pd
import nltk
import spacy
from spacy import displacy 
import tqdm
import operator
import requests
from bs4 import BeautifulSoup
import re
import pt_core_news_sm
import en_core_web_sm
from collections import Counter
import random
import operator
import matplotlib.pyplot as plt

/usr/local/lib/python3.8/dist-packages/spacy/util.py:271: UserWarning: [W031] Model 'pt_core_news_sm' (3.4.0) requires spaCy v3.4 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.8/dist-packages/spacy/util.py:271: UserWarning: [W031] Model 'en_core_web_sm' (3.4.1) requires spaCy v3.4 and is incompatible with the current spaCy version (2.3.0). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


# Pos tagging

## Abordagem Manual

Em empresas, quando queremos construir/usar um POS Tagging, estamos buscando uma ferramenta para trabalhar em algum cenário específico e direcionado ao negócio. Por exemplo, recuperar todos os verbos de um texto, para analisar de forma geral as ações que um cliente conecta com a marca no Twitter. Nesse sentido, podemos usar duas abordagens:

    1 - Construir um modelo próprio que resolva esse cenário
    2 - Usar modelos que já existem e, se necessário, treiná-los para melhor performance no nosso cenário

Para esse exercício, vamos simular esse cenário de recuperar verbos de um texto. Apesar da abordagem 2 parecer mais interessante, vamos tentar construir nosso próprio modelo e aprender um pouco sobre a complexidade do problema.

Inicialmente, vamos reduzir bem nosso escopo, focando em

    - REGEX com NamedGoups
    - Verbos simples, como estudar
    
Use o método abaixo para verificar quais grupos foram identificados pela expressão regular:

```python
def print_found_groups(pattern, phrase):
    regex = re.compile(pattern)
    for m in regex.finditer(phrase):
        print({k:v for(k,v) in m.groupdict().items() if v is not None})
```

Mais informações de regex: https://docs.python.org/pt-br/3.8/howto/regex.html

In [4]:
def print_found_groups(pattern, phrase):
    regex = re.compile(pattern)
    for m in regex.finditer(phrase):
        print({k:v for(k,v) in m.groupdict().items() if v is not None})

In [5]:
phrase1 = "eu estudo, você estuda, nós estudamos e vc sabe como é... estudos em todo lugar "

In [6]:
estudar_pattern_try1 = r'(?P<VERB_ESTUDAR>\bestud\w?\b)'

In [7]:
print_found_groups(estudar_pattern_try1, phrase1)

{'VERB_ESTUDAR': 'estudo'}
{'VERB_ESTUDAR': 'estuda'}


<b> Atividade </b>

1) Modifique o padrão da regex para conseguir reconhecer todas as conjugações do verbo estudar na 'phrase1'

In [8]:
phrase1 = "eu estudo, você estuda, nós estudamos e vc sabe como é... estudos em todo lugar "

In [9]:
estudar_pattern_try1 = r'(?P<VERB_ESTUDAR>\bestud\w{1,4}\b)'

In [10]:
print_found_groups(estudar_pattern_try1, phrase1)

{'VERB_ESTUDAR': 'estudo'}
{'VERB_ESTUDAR': 'estuda'}
{'VERB_ESTUDAR': 'estudamos'}
{'VERB_ESTUDAR': 'estudos'}


<b> Atividade </b>

2) A solução da atividade 1 trouxe todos verbos? Todas as palavras eram verbos?  Este método é simples?<br>
**R.: Não. Nesse exemplo nem todos são verbos, no caso da palavra "estudo" e "estudos" ambas são substantivo. Apesar de ser uma técnica simples, necessita de um certo cuidado, ao criar os padrões que iremos utilizar.** 

## Mac_Morpho com NLTK

In [11]:
nltk.download('mac_morpho')
print("WORDS ", nltk.corpus.mac_morpho.tagged_words())
print("______________")
print("SENTS ",nltk.corpus.mac_morpho.tagged_sents())

[nltk_data] Downloading package mac_morpho to /root/nltk_data...


WORDS  [('Jersei', 'N'), ('atinge', 'V'), ('média', 'N'), ...]
______________
SENTS  [[('Jersei', 'N'), ('atinge', 'V'), ('média', 'N'), ('de', 'PREP'), ('Cr$', 'CUR'), ('1,4', 'NUM'), ('milhão', 'N'), ('em', 'PREP|+'), ('a', 'ART'), ('venda', 'N'), ('de', 'PREP|+'), ('a', 'ART'), ('Pinhal', 'NPROP'), ('em', 'PREP'), ('São', 'NPROP'), ('Paulo', 'NPROP')], [('Programe', 'V'), ('sua', 'PROADJ'), ('viagem', 'N'), ('a', 'PREP|+'), ('a', 'ART'), ('Exposição', 'NPROP'), ('Nacional', 'NPROP'), ('do', 'NPROP'), ('Zebu', 'NPROP'), (',', ','), ('que', 'PRO-KS-REL'), ('começa', 'V'), ('dia', 'N'), ('25', 'N|AP')], ...]


[nltk_data]   Unzipping corpora/mac_morpho.zip.


In [12]:
words_tagged = nltk.corpus.mac_morpho.tagged_words()

<b> Atividade </b>

3) Gere a quantidade de itens de cada tag no mac_morpho

In [13]:
# Utilizei o len para contar o total de tags
len(words_tagged)

1170095

In [14]:
# Criei um contador para saber o total de cada tag
words_tagged_count = pd.Series(list(words_tagged)).value_counts()

In [15]:
# Para facilitar criei um dataframe 
df = pd.DataFrame (words_tagged_count, columns = ['n_tags'])

4) Quais são as tags mais com maior frequência? E quais com menor? Esse resultado faz sentido?

In [16]:
# Esses são as 10 tags que aparecem com maior frequencia
df.head(10)

,n_tags
"(,, ,)",68491
"(o, ART)",51854
"(a, ART)",46588
"(de, PREP)",43093
"(de, PREP|+)",39427
"(e, KC)",21267
"("", "")",21069
"(em, PREP|+)",18586
"(os, ART)",14541
"(em, PREP)",11826


In [17]:
# Abaixo fiz um tail para printar as menores frequências de tag
df.tail(10)

,n_tags
"(2.990, NUM)",1
"(820-7747, N|TEL)",1
"(851-0330, N|TEL)",1
"(Form, NPROP)",1
"(adeptas, ADJ)",1
"(289-8281, N|TEL)",1
"(Super, N)",1
"(placa, N|EST)",1
"(851-5151, N|TEL)",1
"(treinopara, V)",1


## POS Tagging com Spacy - Desambiguando frases

Uma das aplicações do pos tagging é auxiliar na desambiguação de palavras. Uma mesma palavra pode desempenhar funções semântias diferentes, dependendo do contexto.

<b> Atividade </b>

5) Considere as frases abaixo:

5.1) frases grupo1:
```python
frase0="Ele foi andar"
frase1="O andar estava triste"
```

5.2) frase grupo2:
```python
frase2="Essa cola cola papel"
```
5.3) frases grupo3:
```python
frase3="Hora de fazer imposto de renda"
frase4="Aquilo foi imposto pelo chefe"
```

5.4) frases grupo4:
```python
frase5="Hora de fazer imposto de renda"
frase6="Aquilo foi imposto pelo chefe"
```

5.5) frases grupo5:
```python
frase7="Todo dia eu caminho"
frase8="Esse é o caminho mais fácil"
```

Utilizando o modelo ``pt_core_news_sm`` da biblioteca spacy faça as seguinte análises. Para cada token:

- a) imprima a função sintática (pos-tagging) em texto
- b) relação de dependência sintática em texto
- c) gera uma imagem das relações acims utilizando o método abaixo:
- d) análise a palavra em comum, descreva a diferença sintática encontrada e responda se é gramaticamente correta

```python
#Para imprimir detalhes
displacy.render(doc, style='dep',jupyter=True)

```

Dica: https://spacy.io/api/annotation

In [18]:
import spacy
from spacy.lang.pt.examples import sentences 


<b> Atividade </b>

5.1 Frases Grupo 1

In [19]:
txt0="Ele foi andar"
txt1="O andar estava triste"

In [22]:
model = pt_core_news_sm.load()

KeyError: ignored

In [21]:
txt0_nlp = model(txt0)
txt1_nlp = model(txt1)

NameError: ignored

In [ ]:
for token in txt0_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
for token in txt1_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt0_nlp, style='dep',jupyter=True)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt1_nlp, style='dep',jupyter=True)

<b> d) Disucussão Grupo 1: </b>

<b> Atividade </b>

5.2. Frase Grupo 2

In [ ]:
txt2="Essa cola cola papel"

In [ ]:
txt2_nlp = model(txt2)

In [ ]:
for token in txt0_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt2_nlp, style='dep',jupyter=True)

<b> d) Disucussão Grupo 2: </b>

<b> Atividade </b>

5.3. Frases Grupo 3

In [ ]:
txt3="Segundo o que me disseram, ele estava triste"
txt4="No segundo dia de trabalho ela gastou menos"

In [ ]:
# A seguir rodo o modelo nas duas frases txt3 e txt4
txt3_nlp = model(txt3)
txt4_nlp = model(txt4)

In [ ]:
# 
for token in txt3_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
for token in txt4_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt3_nlp, style='dep',jupyter=True)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt4_nlp, style='dep',jupyter=True)

<b> d) Disucussão Grupo 3: </b>

<b> Atividade </b>

5.4. Frases Grupo 4

In [ ]:
txt5="Hora de fazer imposto de renda"
txt6="Aquilo foi imposto pelo chefe"

In [ ]:
# A seguir rodo o modelo nas duas frases txt5 e txt6
txt5_nlp = model(txt5)
txt6_nlp = model(txt6)

In [ ]:
for token in txt5_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
for token in txt6_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt5_nlp, style='dep',jupyter=True)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt6_nlp, style='dep',jupyter=True)

<b> d) Disucussão Grupo 4: </b>

<b> Atividade </b>

5.5. Frases Grupo 5

In [ ]:
txt7="Todo dia eu caminho"
txt8="Esse é o caminho mais fácil"

In [ ]:
# A seguir rodo o modelo nas duas frases txt7 e txt8
txt7_nlp = model(txt7)
txt8_nlp = model(txt8)

In [ ]:
# 
for token in txt7_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# 
for token in txt8_nlp:
  print(token.text,token.pos,token.dep_)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt7_nlp, style='dep',jupyter=True)

In [ ]:
# Representa graficamente a relação de dependência entre as palavras
displacy.render(txt8_nlp, style='dep',jupyter=True)

<b> d) Disucussão Grupo 5: </b>

<b> Esse modelo do spacy para português, em 2020 tinha acurácia de 79,94 pos-tagging. No intervalo de 2 anos o modelo evoluiu e apresenta 96% de acurácia</b>


# Named Entity Recognition

## Recomendação de sites

<b> Atividade </b>

6) Capture o texto de cada notícia abaixo:

- 0 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml
- 1 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml
- 2 -> https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/04/ultimas-noticias-de-coronavirus-de-4-de-marco.ghtml
- 3 -> https://g1.globo.com/economia/noticia/2020/03/04/projetos-reduzem-em-r-96-bilhoes-emendas-parlamentares-no-orcamento-2020-diz-governo.ghtml
- 4 -> https://g1.globo.com/politica/noticia/2020/03/04/documento-liga-assessor-de-eduardo-bolsonaro-a-conta-em-rede-social-para-ataques-pessoais.ghtml
- 5 -> https://g1.globo.com/rj/rio-de-janeiro/noticia/2020/03/04/flavio-bolsonaro-entra-com-recurso-para-tentar-paralisar-investigacao-sobre-suspeita-de-rachadinha.ghtml
- 6 -> https://g1.globo.com/economia/noticia/2020/03/04/pib-do-brasil-cresce-11percent-em-2019.ghtml
- 7 -> https://g1.globo.com/economia/noticia/2020/03/04/tudo-dentro-do-previsto-diz-paulo-guedes-sobre-resultado-do-pib.ghtml
- 8 -> https://g1.globo.com/sp/santos-regiao/noticia/2020/03/04/sobe-numero-de-mortos-apos-temporal-na-baixada-santista.ghtml
- 9 -> https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml

Para isto utilize o código abaixo:
```python
def match_class(target):                                                        
    def do_match(tag):                                                          
        classes = tag.get('class', [])                                          
        return all(c in classes for c in target)                                
    return do_match 

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text
```

In [23]:
def match_class(target):                                                        
    def do_match(tag):                                                          
        classes = tag.get('class', [])                                          
        return all(c in classes for c in target)                                
    return do_match 

def get_text_url(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    #remove marcações de scripts e style
    texto = soup.find_all(match_class(["content-text__container"]))
    all_text = ""
    for t in texto:
        all_text += t.get_text()
    return all_text

In [24]:
get_text_url('https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml')

' A Secretaria Estadual de Saúde de São Paulo divulgou nesta terça-feira (3) que caiu o número de pacientes com suspeita de coronavírus no estado. Enquanto na segunda-feira (2) eram 163 casos suspeitos, nesta terça são 130.  Foram descartados, ao todo, 104 casos no estado, destes, 56 foram nas últimas 24 horas. Entretanto, segundo informou a pasta, entre segunda (2) e terça (3), 23 novos surgiram.  Permanecem confirmados apenas 2 casos da doença no país, ambos no estado de São Paulo. Os pacientes estão em quarentena domiciliar. Ambos estiveram em viagem na Itália.  "Os dois confirmados estão evoluindo bem clinicamente, mas ainda têm alguns sintomas que fazem com que eles continuem em isolamento domiciliar", salientou Paulo Menezes, coordenador do comitê de operações emergenciais (COE) da Secretaria Estadual de Saúde.  O número de pessoas que tiveram contato com o segundo caso confirmado não foi divulgado pela secretaria. No primeiro caso confirmado, eram 34 pessoas, entre passageiros d

In [25]:
noticias = {}
noticias[0] = 'https://g1.globo.com/sp/sao-paulo/noticia/2020/03/03/cai-para-130-numero-de-casos-suspeitos-de-coronavirus-no-estado-de-sao-paulo.ghtml'
noticias[1] = 'https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml'
noticias[2] = 'https://g1.globo.com/bemestar/coronavirus/noticia/2020/03/04/ultimas-noticias-de-coronavirus-de-4-de-marco.ghtml'
noticias[3] = 'https://g1.globo.com/economia/noticia/2020/03/04/projetos-reduzem-em-r-96-bilhoes-emendas-parlamentares-no-orcamento-2020-diz-governo.ghtml'
noticias[4] = 'https://g1.globo.com/politica/noticia/2020/03/04/documento-liga-assessor-de-eduardo-bolsonaro-a-conta-em-rede-social-para-ataques-pessoais.ghtml'
noticias[5] = 'https://g1.globo.com/rj/rio-de-janeiro/noticia/2020/03/04/flavio-bolsonaro-entra-com-recurso-para-tentar-paralisar-investigacao-sobre-suspeita-de-rachadinha.ghtml'
noticias[6] = 'https://g1.globo.com/economia/noticia/2020/03/04/pib-do-brasil-cresce-11percent-em-2019.ghtml'
noticias[7] = 'https://g1.globo.com/economia/noticia/2020/03/04/tudo-dentro-do-previsto-diz-paulo-guedes-sobre-resultado-do-pib.ghtml'
noticias[8] = 'https://g1.globo.com/sp/santos-regiao/noticia/2020/03/04/sobe-numero-de-mortos-apos-temporal-na-baixada-santista.ghtml'
noticias[9] = 'https://g1.globo.com/sp/sao-paulo/noticia/2020/03/04/ministerio-da-saude-confirma-terceiro-paciente-com-coronavirus-em-sp-um-quarto-caso-aguarda-contraprova.ghtml'

In [26]:
texto_noticias = {id:get_text_url(noticias[id]) for id in noticias}

<b> Atividade </b>

7) Capture as entidades de cada uma das notícias utilizando o modelo ``pt_core_news_sm`` do spacy.

<i> Dica: ents retorna as entidades de um texto </i>

In [30]:
pip install scispacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 44 kB 1.7 MB/s 
     |████████████████████████████████| 13.4 MB 8.5 MB/s 
     |████████████████████████████████| 71 kB 9.2 MB/s 
     |████████████████████████████████| 6.6 MB 38.9 MB/s 
     |████████████████████████████████| 188 kB 75.3 MB/s 
     |████████████████████████████████| 819 kB 36.9 MB/s 
     |████████████████████████████████| 492 kB 44.5 MB/s 
     |████████████████████████████████| 10.2 MB 64.4 MB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.2
    Uninstalling catalogue-1.0.2:
      Successfully uninstalled catalogue-1.0.2
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.6
    Uninstalling srsly-1.0.6:
      Successfully uninstalled srsly-1.0.6
  Attempting uninstall: blis
    Found existing installation: blis 0.4.1
    Uninstalling blis-0.4.1:
      Successfully uninsta

In [2]:
from spacy.lang.en import English

ValueError: ignored

In [27]:
model = pt_core_news_sm.load()

KeyError: ignored

In [28]:
not0 = model(texto_noticias[0])

NameError: ignored

In [ ]:
# Verificando se o tipo da noticia na posição zero é um spacy.tokens.doc.Doc
# Positivo, foi tokenizado e não é mais uma string
type(not0)

In [ ]:
# Utilizei o not0.ents e ele traz todos os tokens gerados - todas as entidades
# Nesse caso, geramos para a notícia zero, precisamos gerar de todas as notícias
not0.ents

In [ ]:
texto_noticias_nlp = {id:model(texto_noticias[id]) for id in noticias}

In [ ]:
texto_noticias_nlp[2]

In [ ]:
entidades = {id:texto_noticias_nlp[id].ents for id in noticias}

In [ ]:
# Imprimindo todas as entidades
entidades

In [ ]:
# Depois dessa etapa a próxima é de transformar todas as entidades em string
entidades = {id:set(map(str,texto_noticias_nlp[id].ents)) for id in noticias}

In [ ]:
entidades

<b> Atividade </b>

8) Quais são as 5 entidades que mais apareceram nas notíciais? Critique esse resultado, ele faz sentido?

In [ ]:
todas_entidades = []

In [ ]:
for id in entidades:
  todas_entidades.extend(entidades[id])
Counter(todas_entidades)

<b> Atividade </b>

9) Para a notítica 0, imprima quais foram os tipos das entidades encontradas e plote um gráfico com esses dados.
Faça uma análise desse resultado

<i> Dica: label_ retorna o nome da entidade </i>

In [ ]:
for palavra in texto_noticias_nlp[0].ents:
  print(palavra, palavra.label_)

In [ ]:
labels = []
for palavra in texto_noticias_nlp[0].ents:
  labels.append(palavra.label_)
  print(palavra,palavra.label_)

In [ ]:
Counter(labels)

<b> Atividade </b>

10) Plote um gráfico com esses dados com todos os tipos das entidades de todas as notícias. Compare este resultado com o resultado da notícia 0.

<b> Atividade </b>

11) Imprima a notícia 0, colorindo as entidades encontradas, para isto utilize o método abaixo:

```python
displacy.render(doc, jupyter=True, style='ent')
```

<i> Dica: entidades que existem no spacy https://spacy.io/api/annotation#section-named-entities 

Caso tenha dúvida sobre alguma marcalçao use:</i>

```python
spacy.explain("MISC")
```

In [ ]:
displacy.render(texto_noticias_nlp[0], jupyter=True, style='ent')

<b> Atividade </b>

12) Seja o documento 0 sua referência. Calcule a similaridade das entidades do documento 0 com todos outros 9. Para calcular a similaridade utilize o método jacard (implementado na aula 3). 

In [ ]:
# Método Jacard = O valor da intercessão /(dividido pela) Qte da união
# Se o valor for igual a um, é pq os documentos são idênticos ou possui
# As mesmas entidades
def jacard_similarity(list1, list2):
  s1 = set(list1)
  s2 = set(list2)
  return len(s1.intersection(s2)) / len(s1.union(s2))

In [ ]:
print('Utilizando o método Jacard a similaridade entre o documento 0 e 1 é de',jacard_similarity(entidades[0],entidades[1]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 2 é de',jacard_similarity(entidades[0],entidades[2]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 3 é de',jacard_similarity(entidades[0],entidades[3]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 4 é de',jacard_similarity(entidades[0],entidades[4]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 5 é de',jacard_similarity(entidades[0],entidades[5]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 6 é de',jacard_similarity(entidades[0],entidades[6]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 7 é de',jacard_similarity(entidades[0],entidades[7]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 8 é de',jacard_similarity(entidades[0],entidades[8]))
print('Utilizando o método Jacard a similaridade entre o documento 0 e 9 é de',jacard_similarity(entidades[0],entidades[9]))

<b> Atividade </b>

13) Com os cálculos da questão anterior, responda: quais foram os três documentos com maior similaridade? Eles posssuem conteúdos semelhantes? E qual o documento com menor similaridade? Este documento faz sentido?<br>
**R.: De todos os documentos comparados com zero, os documentos que obtiveram maior similaridade são os documentos 1 e 9. E os que apresentaram as menores valores de similaridades foram os documentos 2 e 8.**

## Retreino NER

Aqui nós iremos ilustrar como treinar um modelo NER no spacy

In [ ]:
# Fazendo o carregamento do modelo
nlp = pt_core_news_sm.load()

In [ ]:
texto = "João nasceu em Paris em 01/01/2000"

In [ ]:
texto_nlp = nlp(texto)

In [ ]:
displacy.render(texto_nlp, style='ent',jupyter=True)

Observe que o modelo identifica <b> João </b> como pessoa e <b> Paris </b> como localidade.

Apesar do modelo ter identificado essas duas entidades ele naõ identificou a <b> data </b>.

Assim, aqui iremos mostrar como retreinar o modelo para capturar também a data.


In [ ]:
#dados de treino. Note que para cada frase, a posição de uma determinada entidade.
train_data = [
("Em 04/12/1992 nasceu Joana", {'entities':[(3, 13, "DATE"), (21,26, "PER")]}),
("Data de início: 10/01/2018", {'entities':[(16, 26, "DATE")]}),
("Maria se mudou para Paris", {'entities':[(20,26, "LOC")]}),
("Paris cidade das luzes", {'entities':[(0,5, "LOC")]}),    
("Maria nasceu em Contagem no dia 07/05/2018", {'entities':[(0,5,"PER"),(16,24,"LOC"),(32,42, "DATE")]})
]

### Retreino

- explosion/spaCy/master/examples/training/train_ner.py -> O spcay disponibiliza esse código para treinamentos.
- Mais informações? https://spacy.io/usage/training

In [ ]:
def train_spacy(data, iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('pt')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)           
    return nlp

In [ ]:
modelo_novo = train_spacy(train_data, 30)

In [ ]:
modelo_novo.to_disk("modelo_novo_NER")

<b> Atividade </b>

14) O ``modelo_novo`` já foi criado. Aplique o texto em que demonstramos que a data não foi identificada e depois imprima utilizando o ``displacy``.
O problema foi resolvido? E se mudarmos o formato da data, ele reconhce? Quais outras entidades o modelo não consegue reconhecer? 

In [ ]:
texto_nlp = modelo_novo(texto)
displacy.render(texto_nlp, style='ent',jupyter=True)